# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 4 2022 8:16AM,250033,19,ADV80007383,"AdvaGen Pharma, Ltd",Released
1,Nov 4 2022 8:07AM,250031,10,0086142755,ISDIN Corporation,Released
2,Nov 4 2022 8:07AM,250031,10,0086142757,ISDIN Corporation,Released
3,Nov 4 2022 8:07AM,250031,10,0086142758,ISDIN Corporation,Released
4,Nov 4 2022 8:07AM,250031,10,0086142761,ISDIN Corporation,Released
5,Nov 4 2022 8:07AM,250031,10,0086142762,ISDIN Corporation,Released
6,Nov 4 2022 8:07AM,250031,10,0086142760,ISDIN Corporation,Released
7,Nov 4 2022 8:07AM,250031,10,0086142795,ISDIN Corporation,Released
8,Nov 4 2022 8:07AM,250031,10,0086142800,ISDIN Corporation,Released
9,Nov 4 2022 8:07AM,250031,10,0086142813,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,250028,Released,3
7,250029,Released,4
8,250030,Released,1
9,250031,Released,14
10,250033,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250028,NaN,3.0
250029,NaN,4.0
250030,NaN,1.0
250031,NaN,14.0
250033,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249984,1.0,0.0
249985,0.0,1.0
249992,0.0,1.0
250010,0.0,1.0
250022,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249984,1,0
249985,0,1
249992,0,1
250010,0,1
250022,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249984,1,0
1,249985,0,1
2,249992,0,1
3,250010,0,1
4,250022,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249984,1,
1,249985,,1
2,249992,,1
3,250010,,1
4,250022,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 4 2022 8:16AM,250033,19,"AdvaGen Pharma, Ltd"
1,Nov 4 2022 8:07AM,250031,10,ISDIN Corporation
15,Nov 4 2022 8:02AM,250030,10,"Snap Medical Industries, LLC"
16,Nov 4 2022 8:01AM,250029,10,Emerginnova
20,Nov 4 2022 7:45AM,250028,19,"AdvaGen Pharma, Ltd"
23,Nov 3 2022 4:33PM,250025,12,LF of America Corp.
24,Nov 3 2022 3:52PM,250022,18,Innova Softgel LLC
25,Nov 3 2022 2:43PM,250010,19,ACG North America LLC
26,Nov 3 2022 1:53PM,249992,20,"HVL, LLC dba Atrium Innovations"
27,Nov 3 2022 12:38PM,249985,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 4 2022 8:16AM,250033,19,"AdvaGen Pharma, Ltd",,1
1,Nov 4 2022 8:07AM,250031,10,ISDIN Corporation,,14
2,Nov 4 2022 8:02AM,250030,10,"Snap Medical Industries, LLC",,1
3,Nov 4 2022 8:01AM,250029,10,Emerginnova,,4
4,Nov 4 2022 7:45AM,250028,19,"AdvaGen Pharma, Ltd",,3
5,Nov 3 2022 4:33PM,250025,12,LF of America Corp.,,1
6,Nov 3 2022 3:52PM,250022,18,Innova Softgel LLC,,1
7,Nov 3 2022 2:43PM,250010,19,ACG North America LLC,,1
8,Nov 3 2022 1:53PM,249992,20,"HVL, LLC dba Atrium Innovations",,1
9,Nov 3 2022 12:38PM,249985,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 8:16AM,250033,19,"AdvaGen Pharma, Ltd",1,
1,Nov 4 2022 8:07AM,250031,10,ISDIN Corporation,14,
2,Nov 4 2022 8:02AM,250030,10,"Snap Medical Industries, LLC",1,
3,Nov 4 2022 8:01AM,250029,10,Emerginnova,4,
4,Nov 4 2022 7:45AM,250028,19,"AdvaGen Pharma, Ltd",3,
5,Nov 3 2022 4:33PM,250025,12,LF of America Corp.,1,
6,Nov 3 2022 3:52PM,250022,18,Innova Softgel LLC,1,
7,Nov 3 2022 2:43PM,250010,19,ACG North America LLC,1,
8,Nov 3 2022 1:53PM,249992,20,"HVL, LLC dba Atrium Innovations",1,
9,Nov 3 2022 12:38PM,249985,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 8:16AM,250033,19,"AdvaGen Pharma, Ltd",1,
1,Nov 4 2022 8:07AM,250031,10,ISDIN Corporation,14,
2,Nov 4 2022 8:02AM,250030,10,"Snap Medical Industries, LLC",1,
3,Nov 4 2022 8:01AM,250029,10,Emerginnova,4,
4,Nov 4 2022 7:45AM,250028,19,"AdvaGen Pharma, Ltd",3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 8:16AM,250033,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Nov 4 2022 8:07AM,250031,10,ISDIN Corporation,14.0,NaN
2,Nov 4 2022 8:02AM,250030,10,"Snap Medical Industries, LLC",1.0,NaN
3,Nov 4 2022 8:01AM,250029,10,Emerginnova,4.0,NaN
4,Nov 4 2022 7:45AM,250028,19,"AdvaGen Pharma, Ltd",3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 4 2022 8:16AM,250033,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Nov 4 2022 8:07AM,250031,10,ISDIN Corporation,14.0,0.0
2,Nov 4 2022 8:02AM,250030,10,"Snap Medical Industries, LLC",1.0,0.0
3,Nov 4 2022 8:01AM,250029,10,Emerginnova,4.0,0.0
4,Nov 4 2022 7:45AM,250028,19,"AdvaGen Pharma, Ltd",3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,750090,19.0,0.0
12,250025,1.0,0.0
18,250022,1.0,0.0
19,750071,5.0,0.0
20,249992,1.0,0.0
21,499969,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,750090,19.0,0.0
1,12,250025,1.0,0.0
2,18,250022,1.0,0.0
3,19,750071,5.0,0.0
4,20,249992,1.0,0.0
5,21,499969,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,19.0,0.0
1,12,1.0,0.0
2,18,1.0,0.0
3,19,5.0,0.0
4,20,1.0,0.0
5,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,12,Released,1.0
2,18,Released,1.0
3,19,Released,5.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,18,19,20,21
Status,,,,,,
Executing,0.0,0.0,0.0,0.0,0.0,1.0
Released,19.0,1.0,1.0,5.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,18,19,20,21
0,Executing,0.0,0.0,0.0,0.0,0.0,1.0
1,Released,19.0,1.0,1.0,5.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,18,19,20,21
0,Executing,0.0,0.0,0.0,0.0,0.0,1.0
1,Released,19.0,1.0,1.0,5.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()